In [1]:
import requests
import tensorflow as tf
import json
import base64
from pprint import PrettyPrinter
import pandas as pd

In [2]:
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/fake-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1740567021'}]}


In [3]:
df = pd.read_csv("data/News.csv")

df.head()

,text,label
0,sing praise mahinda lose voice sri lanka disro...,1
1,kojc rally first time davao city appoint son god,1
2,bill gate people resist mrna tsunami exclude s...,1
3,cure covid19 forward budesonide asthma drug pl...,1
4,president trump form patriot party 81 day elec...,1


In [4]:
test_text = df["text"][0]

test_text

'sing praise mahinda lose voice sri lanka disrobe go japan fun good gnanasaera thera'

In [5]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])
        )
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps({"signature_name": "serving_default", "instances": result})

In [6]:
def make_predictions(text):
    json_data = prepare_json(text)

    endpoint = "http://localhost:8080/v1/models/fake-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return prediction

In [7]:
make_predictions(test_text)

0.999989092

In [8]:
df["label"][0]

1